# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json 

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Create a reference the CSV file desired
csv_path = "../output_data/weatherpy_data.csv"

# Read the CSV into a Pandas DataFrame
weather_df = pd.read_csv(csv_path)

# Print the first five rows of data to the screen
weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,monte patria,0,FR,1617067996,81,42.4687,9.3916,54.0,5.75
1,buraydah,0,SA,1617067996,31,26.3260,43.9750,59.0,12.66
2,albany,0,US,1617067708,35,42.6001,-73.9662,39.0,1.01
3,hobart,0,AU,1617067997,48,-42.8794,147.3294,66.0,10.36
4,ushuaia,20,AR,1617067888,76,-54.8000,-68.3000,50.0,7.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
#drop NaN values
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations 
locations = weather_df[["Lat", "Lng"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#ideal weather data frame
filtered_weather_df = weather_df

# Drop any rows that don't contain all three conditions

# A max temperature lower than 80/ higher than 70
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp"] < 80) & (filtered_weather_df["Max Temp"] > 70)]

#wind less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 10]

#no clouds
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]

#drop null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
31,saint-philippe,0,RE,1617068004,73,-21.3585,55.7679,77.00,5.75
109,saint-joseph,0,RE,1617068025,73,-21.3667,55.6167,77.00,5.75
226,belmonte,0,BR,1617068054,78,-15.8631,-38.8828,77.00,4.61
277,planeta rica,0,BR,1617068068,83,-20.2934,-40.3970,78.80,6.91
311,kibala,0,TD,1617068078,25,9.1104,18.3463,78.80,6.91
413,luzhou,0,CN,1617068106,58,28.8724,105.4405,77.59,1.03
441,khartoum,0,SD,1617068063,17,15.5518,32.5324,73.00,1.99
443,narrabri,0,AU,1617068114,47,-30.3167,149.7833,79.00,6.91
503,dubbo,0,AU,1617068130,56,-32.2500,148.6167,73.40,9.22
506,saint-pierre,0,RE,1617068131,73,-21.3393,55.4781,77.00,5.75


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = filtered_weather_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 31: saint-philippe.
Closest hotel is Chambres d'hôte "La Trinité".
------------
Retrieving Results for Index 109: saint-joseph.
Closest hotel is "Plantation Bed and Breakfast.
------------
Retrieving Results for Index 226: belmonte.
Closest hotel is Pousada Monte Carmelo O Pão.
------------
Retrieving Results for Index 277: planeta rica.
Closest hotel is Classic Motel.
------------
Retrieving Results for Index 311: kibala.
Missing field/result... skipping.
------------
Retrieving Results for Index 413: luzhou.
Closest hotel is Shijichun Hotel.
------------
Retrieving Results for Index 441: khartoum.
Closest hotel is Hotel China.
------------
Retrieving Results for Index 443: narrabri.
Closest hotel is Crossroads Hotel.
------------
Retrieving Results for Index 503: dubbo.
Closest hotel is Comfort Inn Blue Lagoon.
------------
Retrieving Results for Index 506: saint-pierre.
Closest hotel is Lindsey Hôtel.
------------
Retrieving Results for Index 509: pahrum

In [6]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
31,saint-philippe,0,RE,1617068004,73,-21.3585,55.7679,77.00,5.75,"Chambres d'hôte ""La Trinité"""
109,saint-joseph,0,RE,1617068025,73,-21.3667,55.6167,77.00,5.75,"""Plantation Bed and Breakfast"
226,belmonte,0,BR,1617068054,78,-15.8631,-38.8828,77.00,4.61,Pousada Monte Carmelo O Pão
277,planeta rica,0,BR,1617068068,83,-20.2934,-40.3970,78.80,6.91,Classic Motel
311,kibala,0,TD,1617068078,25,9.1104,18.3463,78.80,6.91,NaN
413,luzhou,0,CN,1617068106,58,28.8724,105.4405,77.59,1.03,Shijichun Hotel
441,khartoum,0,SD,1617068063,17,15.5518,32.5324,73.00,1.99,Hotel China
443,narrabri,0,AU,1617068114,47,-30.3167,149.7833,79.00,6.91,Crossroads Hotel
503,dubbo,0,AU,1617068130,56,-32.2500,148.6167,73.40,9.22,Comfort Inn Blue Lagoon
506,saint-pierre,0,RE,1617068131,73,-21.3393,55.4781,77.00,5.75,Lindsey Hôtel


In [7]:
# Assign the marker layer to a variable
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))